# Install deequ

In [1]:
import $ivy.`com.amazon.deequ:deequ:2.0.1-spark-3.2`

import $ivy.$                                       

## spark setup

In [2]:
import org.apache.log4j.Logger
import org.apache.log4j.Level

Logger.getLogger("org").setLevel(Level.OFF)
Logger.getLogger("akka").setLevel(Level.OFF)

import org.apache.log4j.Logger

import org.apache.log4j.Level



In [3]:
import org.apache.spark.sql._

val spark = SparkSession.builder()
      .master("local")
      .appName("test")
//       .config("spark.ui.enabled", "false")
      .getOrCreate()
    spark.sparkContext.setCheckpointDir(System.getProperty("java.io.tmpdir"))
import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@4c7d0997
import spark.implicits._

In [4]:
import org.apache.spark.sql.DataFrame
import almond.interpreter.api.DisplayData

def show(df:DataFrame) {
    val header = df.columns.map( columnName => s"<td><b>${columnName.toString}<b></td>").mkString("")
    val tableRows = df.collect.map(r => {
        r.toSeq.map(value =>s"<td>$value</td>").mkString("")
    }).map(trow=> s"<tr>$trow<tr>").mkString("")
    val htmlTable = s"<html><table><tr>${header}<tr>$tableRows</table></html>"

    display(DisplayData(
    Map(
      "text/html" -> htmlTable)))
}

import org.apache.spark.sql.DataFrame

import almond.interpreter.api.DisplayData


defined function show

## read data

Both hour.csv and day.csv have the following fields, except hr which is not available in day.csv

- instant: record index
- dteday : date
- season : season (1:winter, 2:spring, 3:summer, 4:fall)
- yr : year (0: 2011, 1:2012)
- mnth : month ( 1 to 12)
- hr : hour (0 to 23)
- holiday : weather day is holiday or not (extracted from [Web Link])
- weekday : day of the week
- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
+ weathersit :
- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- temp : Normalized temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-8, t_max=+39 (only in hourly scale)
- atemp: Normalized feeling temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-16, t_max=+50 (only in hourly scale)
- hum: Normalized humidity. The values are divided to 100 (max)
- windspeed: Normalized wind speed. The values are divided to 67 (max)
- casual: count of casual users
- registered: count of registered users
- cnt: count of total rental bikes including both casual and registered

In [5]:
import com.amazon.deequ.suggestions.{ConstraintSuggestionRunner, Rules}
import org.apache.spark.sql.functions._


import com.amazon.deequ.analyzers.Sum
import com.amazon.deequ.anomalydetection.BatchNormalStrategy
import com.amazon.deequ.checks.CheckLevel
import com.amazon.deequ.repository.ResultKey
import com.amazon.deequ.repository.fs.FileSystemMetricsRepository
import com.amazon.deequ.{AnomalyCheckConfig, VerificationResult, VerificationSuite}
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions.col

import java.nio.file.Paths
import java.time.format.DateTimeFormatter
import java.time.temporal.ChronoUnit
import java.time.{Duration, Instant, ZoneId}


val by1w = Window.partitionBy("yr").rowsBetween(Window.currentRow - 7, Window.currentRow)
val df: DataFrame = spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load("data/day.csv")
  .withColumn("avg1w", avg(col("cnt")).over(by1w))
df.printSchema
show(df.limit(30))

root
 |-- instant: integer (nullable = true)
 |-- dteday: string (nullable = true)
 |-- season: integer (nullable = true)
 |-- yr: integer (nullable = true)
 |-- mnth: integer (nullable = true)
 |-- holiday: integer (nullable = true)
 |-- weekday: integer (nullable = true)
 |-- workingday: integer (nullable = true)
 |-- weathersit: integer (nullable = true)
 |-- temp: double (nullable = true)
 |-- atemp: double (nullable = true)
 |-- hum: double (nullable = true)
 |-- windspeed: double (nullable = true)
 |-- casual: integer (nullable = true)
 |-- registered: integer (nullable = true)
 |-- cnt: integer (nullable = true)
 |-- avg1w: double (nullable = true)



instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,avg1w
1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985,985.0
2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801,893.0
3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349,1045.0
4,2011-01-04,1,0,1,0,2,1,1,0.2,0.212122,0.590435,0.160296,108,1454,1562,1174.25
5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.22927,0.436957,0.1869,82,1518,1600,1259.4
6,2011-01-06,1,0,1,0,4,1,1,0.204348,0.233209,0.518261,0.0895652,88,1518,1606,1317.1666666666667
7,2011-01-07,1,0,1,0,5,1,2,0.196522,0.208839,0.498696,0.168726,148,1362,1510,1344.7142857142858
8,2011-01-08,1,0,1,0,6,0,2,0.165,0.162254,0.535833,0.266804,68,891,959,1296.5
9,2011-01-09,1,0,1,0,0,0,1,0.138333,0.116175,0.434167,0.36195,54,768,822,1276.125
10,2011-01-10,1,0,1,0,1,1,1,0.150833,0.150888,0.482917,0.223267,41,1280,1321,1341.125


import com.amazon.deequ.suggestions.{ConstraintSuggestionRunner, Rules}

import org.apache.spark.sql.functions._



import com.amazon.deequ.analyzers.Sum

import com.amazon.deequ.anomalydetection.BatchNormalStrategy

import com.amazon.deequ.checks.CheckLevel

import com.amazon.deequ.repository.ResultKey

import com.amazon.deequ.repository.fs.FileSystemMetricsRepository

import com.amazon.deequ.{AnomalyCheckConfig, VerificationResult, VerificationSuite}

import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.functions.col


import java.nio.file.Paths

import java.time.format.DateTimeFormatter

import java.time.temporal.ChronoUnit

import java.time.{Duration, Instant, ZoneId}



by1w: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@17a1953f
df: DataFrame = [instant: int, dteday: string ... 15 more fields]

In [6]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.0`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._


import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._


In [7]:
val (x, y,z) = df.collect.map(r=>(r(1).toString, r(15).toString.toFloat, r(16).toString.toFloat)).toList.unzip3
// val (a, b) = df.collect.map(r=>(r(1).toString, r(16).toString.toFloat)).toList.unzip
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)
val trace1 = Scatter(
  x,
  y,
  name="cnt"
)
val trace2 = Scatter(
  x,
  z,
  name="avg7days"
    
)
plot(Seq(trace1, trace2))

x: List[String] = List(
  "2011-01-01",
...
y: List[Float] = List(
  985.0F,
...
z: List[Float] = List(
  985.0F,
...
trace1: Scatter = Scatter(
  Some(
...
trace2: Scatter = Scatter(
  Some(
...
res6_4: String = "plot-37b53953-b2a9-4dc0-982a-9bbfe3749b00"

# anomaly checks

In [8]:
val property = "java.io.tmpdir"
val anomalyJsonPath = Paths.get(System.getProperty(property), "anomaly.json").toString
println(s"anomaly json file path: ${anomalyJsonPath}")
val metricsRepository = new FileSystemMetricsRepository(spark, path = anomalyJsonPath)

val formatter = DateTimeFormatter.ofPattern("yyyy-MM-dd HH:mm:ss").withZone(ZoneId.of("Etc/UTC"))
val dayFormatter = DateTimeFormatter.ofPattern("yyyy-MM-dd").withZone(ZoneId.of("Etc/UTC"))

val startDate = Instant.from(formatter.parse("2012-03-10 00:00:00"))
val startTimestamp = startDate.toEpochMilli
val endDate = Instant.from(formatter.parse("2012-05-25 00:00:00"))
val days = ChronoUnit.DAYS.between(startDate, endDate)

anomaly json file path: /tmp/anomaly.json


property: String = "java.io.tmpdir"
anomalyJsonPath: String = "/tmp/anomaly.json"
metricsRepository: FileSystemMetricsRepository = com.amazon.deequ.repository.fs.FileSystemMetricsRepository@2ad886fc
formatter: DateTimeFormatter = Value(YearOfEra,4,19,EXCEEDS_PAD)'-'Value(MonthOfYear,2)'-'Value(DayOfMonth,2)' 'Value(HourOfDay,2)':'Value(MinuteOfHour,2)':'Value(SecondOfMinute,2)
dayFormatter: DateTimeFormatter = Value(YearOfEra,4,19,EXCEEDS_PAD)'-'Value(MonthOfYear,2)'-'Value(DayOfMonth,2)
startDate: Instant = 2012-03-10T00:00:00Z
startTimestamp: Long = 1331337600000L
endDate: Instant = 2012-05-25T00:00:00Z
days: Long = 76L

## RelativeRateOfChangeStrategy

In [17]:
import com.amazon.deequ.anomalydetection.RelativeRateOfChangeStrategy

val dfResults = (0l to days).map(
      delta_date => {
        val date = startDate.plus(Duration.ofDays(delta_date))
        val dateStr = dayFormatter.format(date)
        val ts = date.toEpochMilli
        val resultKey = ResultKey(ts)

        val anomalyCheckErrorConfig = Some(AnomalyCheckConfig(
          level = CheckLevel.Error,
          description = s"relativeRate $dateStr",
          withTagValues = Map.empty,
          afterDate = Some(ts - 7 * 24 * 60 * 60 * 1000),
          beforeDate = Some(ts - 1)
        ))

        val verificationResult = VerificationSuite()
          .onData(df.filter(s"dteday = '${dateStr}'"))
          .useRepository(metricsRepository)
          .saveOrAppendResult(resultKey)
          .addAnomalyCheck(
            RelativeRateOfChangeStrategy(
              maxRateIncrease = Some(1.3),
              maxRateDecrease = Some(0.7),
              order = 1),
            Sum("cnt"),
            anomalyCheckErrorConfig
          )
          .run()
        VerificationResult.checkResultsAsDataFrame(spark, verificationResult)
      }
    ).reduce(_ union _)

import com.amazon.deequ.anomalydetection.RelativeRateOfChangeStrategy


dfResults: DataFrame = [check: string, check_level: string ... 4 more fields]

In [10]:
metricsRepository
      .load()
      .forAnalyzers(Seq(Sum("cnt")))
      .getSuccessMetricsAsDataFrame(spark).orderBy("dataset_date").show

+------+--------+----+------+-------------+
|entity|instance|name| value| dataset_date|
+------+--------+----+------+-------------+
|Column|     cnt| Sum| 985.0|1293840000000|
|Column|     cnt| Sum| 801.0|1293926400000|
|Column|     cnt| Sum|1349.0|1294012800000|
|Column|     cnt| Sum|1562.0|1294099200000|
|Column|     cnt| Sum|1600.0|1294185600000|
|Column|     cnt| Sum|1606.0|1294272000000|
|Column|     cnt| Sum|1510.0|1294358400000|
|Column|     cnt| Sum| 959.0|1294444800000|
|Column|     cnt| Sum| 822.0|1294531200000|
|Column|     cnt| Sum|1321.0|1294617600000|
|Column|     cnt| Sum|1263.0|1294704000000|
|Column|     cnt| Sum|1162.0|1294790400000|
|Column|     cnt| Sum|1406.0|1294876800000|
|Column|     cnt| Sum|1421.0|1294963200000|
|Column|     cnt| Sum|1248.0|1295049600000|
|Column|     cnt| Sum|1204.0|1295136000000|
|Column|     cnt| Sum|1000.0|1295222400000|
|Column|     cnt| Sum| 683.0|1295308800000|
|Column|     cnt| Sum|1650.0|1295395200000|
|Column|     cnt| Sum|1927.0|129

In [18]:
show(dfResults)

check,check_level,check_status,constraint,constraint_status,constraint_message
relativeRate 2012-03-10,Error,Error,"AnomalyConstraint(Sum(cnt,None))",Failure,Can't execute the assertion: requirement failed: There have to be previous results in the MetricsRepository!!
relativeRate 2012-03-11,Error,Success,"AnomalyConstraint(Sum(cnt,None))",Success,
relativeRate 2012-03-12,Error,Success,"AnomalyConstraint(Sum(cnt,None))",Success,
relativeRate 2012-03-13,Error,Success,"AnomalyConstraint(Sum(cnt,None))",Success,
relativeRate 2012-03-14,Error,Success,"AnomalyConstraint(Sum(cnt,None))",Success,
relativeRate 2012-03-15,Error,Success,"AnomalyConstraint(Sum(cnt,None))",Success,
relativeRate 2012-03-16,Error,Success,"AnomalyConstraint(Sum(cnt,None))",Success,
relativeRate 2012-03-17,Error,Error,"AnomalyConstraint(Sum(cnt,None))",Failure,Value: 7836.0 does not meet the constraint requirement!
relativeRate 2012-03-18,Error,Success,"AnomalyConstraint(Sum(cnt,None))",Success,
relativeRate 2012-03-19,Error,Success,"AnomalyConstraint(Sum(cnt,None))",Success,


## BatchNormalStrategy

In [15]:
    val dfResults = (0l to days).map(
      delta_date => {
        val date = startDate.plus(Duration.ofDays(delta_date))
        val dateStr = dayFormatter.format(date)
        val ts = date.toEpochMilli
        val resultKey = ResultKey(ts)

        val anomalyCheckErrorConfig = Some(AnomalyCheckConfig(
          level = CheckLevel.Error,
          description = s"batch $dateStr",
          withTagValues = Map.empty,
          afterDate = Some(ts - 7 * 24 * 60 * 60 * 1000),
          beforeDate = Some(ts - 1)
        ))

        val verificationResult = VerificationSuite()
          .onData(df.filter(s"dteday = '${dateStr}'"))
          .useRepository(metricsRepository)
          .saveOrAppendResult(resultKey)
          .addAnomalyCheck(
            BatchNormalStrategy(Some(3.0), Some(3.0)),
            Sum("cnt"),
            anomalyCheckErrorConfig
          )
          .run()
        VerificationResult.checkResultsAsDataFrame(spark, verificationResult)
      }
    ).reduce(_ union _)

dfResults: DataFrame = [check: string, check_level: string ... 4 more fields]

In [13]:
metricsRepository
      .load()
      .forAnalyzers(Seq(Sum("cnt")))
      .getSuccessMetricsAsDataFrame(spark).orderBy("dataset_date").show()

+------+--------+----+------+-------------+
|entity|instance|name| value| dataset_date|
+------+--------+----+------+-------------+
|Column|     cnt| Sum| 985.0|1293840000000|
|Column|     cnt| Sum| 801.0|1293926400000|
|Column|     cnt| Sum|1349.0|1294012800000|
|Column|     cnt| Sum|1562.0|1294099200000|
|Column|     cnt| Sum|1600.0|1294185600000|
|Column|     cnt| Sum|1606.0|1294272000000|
|Column|     cnt| Sum|1510.0|1294358400000|
|Column|     cnt| Sum| 959.0|1294444800000|
|Column|     cnt| Sum| 822.0|1294531200000|
|Column|     cnt| Sum|1321.0|1294617600000|
|Column|     cnt| Sum|1263.0|1294704000000|
|Column|     cnt| Sum|1162.0|1294790400000|
|Column|     cnt| Sum|1406.0|1294876800000|
|Column|     cnt| Sum|1421.0|1294963200000|
|Column|     cnt| Sum|1248.0|1295049600000|
|Column|     cnt| Sum|1204.0|1295136000000|
|Column|     cnt| Sum|1000.0|1295222400000|
|Column|     cnt| Sum| 683.0|1295308800000|
|Column|     cnt| Sum|1650.0|1295395200000|
|Column|     cnt| Sum|1927.0|129

In [16]:
show(dfResults)

check,check_level,check_status,constraint,constraint_status,constraint_message
batch 2012-03-10,Error,Error,"AnomalyConstraint(Sum(cnt,None))",Failure,Can't execute the assertion: requirement failed: There have to be previous results in the MetricsRepository!!
batch 2012-03-11,Error,Error,"AnomalyConstraint(Sum(cnt,None))",Failure,Value: 4911.0 does not meet the constraint requirement!
batch 2012-03-12,Error,Success,"AnomalyConstraint(Sum(cnt,None))",Success,
batch 2012-03-13,Error,Success,"AnomalyConstraint(Sum(cnt,None))",Success,
batch 2012-03-14,Error,Success,"AnomalyConstraint(Sum(cnt,None))",Success,
batch 2012-03-15,Error,Success,"AnomalyConstraint(Sum(cnt,None))",Success,
batch 2012-03-16,Error,Success,"AnomalyConstraint(Sum(cnt,None))",Success,
batch 2012-03-17,Error,Success,"AnomalyConstraint(Sum(cnt,None))",Success,
batch 2012-03-18,Error,Success,"AnomalyConstraint(Sum(cnt,None))",Success,
batch 2012-03-19,Error,Success,"AnomalyConstraint(Sum(cnt,None))",Success,
